In [ ]:
# import sys
# !{sys.executable} -m pip install imbalanced-learn --user
# !{sys.executable} -m pip install hyperopt --user

In [1]:
import numpy
from time import time
from sklearn.model_selection import GridSearchCV
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from copy import deepcopy

from newsgac import config
from newsgac.genres import genre_codes
from newsgac.learners import learners, LearnerSVC, LearnerNB, LearnerXGB, LearnerGB, LearnerMLP, LearnerRF, LearnerLGBM
from newsgac.pipelines.get_sk_pipeline import get_sk_pipeline
from newsgac.pipelines.utils import report
from sklearn import svm


from newsgac import database
from newsgac.data_sources import DataSource
from newsgac.pipelines import Pipeline

# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
# from imblearn.combine import SMOTEENN, SMOTETomek
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/newsgac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
[d.display_title for d in DataSource.objects.all()]

[u'N2BGS Training',
 u'N2BGS Testing',
 u'Linked NRC (2930)',
 u'BOW + 5 features (N2BGS train)',
 u'BOW + 5 features (N2BGS test)',
 u'nrc-1950-1994-a.txt',
 u'nrc-1950-1994-b.txt',
 u'nrc-1950-1994-c.txt',
 u'nrc-1950-1994-d.txt',
 u'nrc-1950-1994-e.txt',
 u'telegraaf-1950-1994-a.txt',
 u'telegraaf-1950-1994-b.txt',
 u'telegraaf-1950-1994-c.txt',
 u'telegraaf-1950-1994-d.txt',
 u'telegraaf-1950-1994-e.txt',
 u'volkskrant-1950-1995-a.txt',
 u'volkskrant-1950-1995-b.txt',
 u'volkskrant-1950-1995-c.txt',
 u'volkskrant-1950-1995-d.txt',
 u'volkskrant-1950-1995-e.txt',
 u'nrc-1965.txt',
 u'nrc-1985.txt',
 u'BOW + 9 features (N2BGS train)',
 u'BOW + 9 features (N2BGS test)',
 u'Linked NRC (2930/9 features)',
 u'BOW Train unique (9 features)',
 u'BOW Test unique (9 features)',
 u'Linked NRC (unique/9 features)',
 u'BOW Train unique (N3BGS/9 features)',
 u'N3BGS FROG Test',
 u'N3BGS FROG Train',
 u'BOW Test unique (N3BGS/9 features)',
 u'BOW Test unique (N3BGS/9 features/collapsed)',
 u'Link

In [3]:
[d.display_title for d in Pipeline.objects.all()]

[u'dummy frog',
 u'dummy bow',
 u'RF FROG',
 u'SVC FROG',
 u'XGB FROG',
 u'NB FROG',
 u'MLP FROG',
 u'RF BOW',
 u'SVC BOW',
 u'XGB BOW',
 u'NB BOW',
 u'MLP BOW',
 u'SVC LIN BOW',
 u'Erik MLP BOW (with stop-words)',
 u'Erik MLP BOW (5 features) ',
 u'Erik MLP BOW (9 features) ',
 u'Erik SVC BOW (9 features)',
 u'Erik RF BOW (9 features)',
 u'Erik XGB BOW (9 features)',
 u'Erik NB BOW (9 features)',
 u'Erik MLP unique 9 features',
 u'Erik NB unique (9 features)',
 u'Erik XGB unique (9 features)',
 u'Erik SVC unique (9 features)',
 u'Erik RF unique (9 features)',
 u'N3BGS MLP unique 9 features',
 u'N3BGS NB unique 9 features ',
 u'N3BGS SVC unique 9 features ',
 u'N3BGS RF unique 9 features',
 u'N3BGS XGB unique 9 features ',
 u'dummy n3bgs bow',
 u'dummy n3bgs frog',
 u'RF FROG N3BGS',
 u'SVC FROG N3BGS',
 u'XGB FROG N3BGS',
 u'Erik NB N3BGS Collapsed',
 u'Erik MLP N3BGS Collapsed',
 u'Erik MLP Linked NRC (2930 articles)',
 u'Erik RF Linked NRC (2930 articles) ',
 u'Erik NB Linked NRC (2

In [4]:
p = Pipeline.objects.all()[59]
print 'Pipeline: ' + p.display_title
print 'DataSource: ' + p.data_source.display_title
print 'NLP Tool: ' + p.nlp_tool.name
print 'Classifier: ' + p.learner.name
print 'Task status: ' + str(p.task.status)

Pipeline: dummy combi for opt ugs
DataSource: 20190228 unbalanced train size=3443
NLP Tool: Frog + TFIDF
Classifier: Multi-Layer Perceptron
Task status: Status.SUCCESS


In [5]:
len(p.data_source.articles)

3099

In [6]:
test_data_source = DataSource.objects.all()[39]
print 'Testing data source: ' + test_data_source.display_title

Testing data source: 20190228 unbalanced test size=3443


In [7]:
skp = p.get_sk_pipeline()
skp_opt = deepcopy(skp)
skp_opt.steps.pop()
skp_opt.steps

[('CleanOCR', <newsgac.nlp_tools.transformers.CleanOCR at 0x7f7176909110>),
 ('StopWordRemoval',
  <newsgac.nlp_tools.transformers.StopWordRemoval at 0x7f7176909310>),
 ('FeatureExtraction', FeatureUnion(n_jobs=None,
         transformer_list=[('frog', FeatureUnion(n_jobs=None,
         transformer_list=[('Basic', <newsgac.nlp_tools.transformers.ExtractBasicFeatures object at 0x7f7173fd6b50>), ('Quote', <newsgac.nlp_tools.transformers.ExtractQuotes object at 0x7f7173fd6b90>), ('Sentiment', Pipeline(memory=None,
       steps=...\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None))],
         transformer_weights=None)),
 ('RobustScaler',
  RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=False,
         with_scaling=True))]

In [8]:
texts = numpy.array([article.raw_text for article in p.data_source.articles])
labels = numpy.array([article.label for article in p.data_source.articles])

X = skp_opt.fit_transform(texts)

In [9]:
# get the test dataset for optimization accuracy
texts_test = numpy.array([article.raw_text for article in test_data_source.articles])
labels_test = numpy.array([article.label for article in test_data_source.articles])

X_test = skp_opt.transform(texts_test)

In [10]:
X_train = X
y_train = labels
y_test = labels_test

In [11]:
y_test.shape

(344,)

In [13]:
kernel_list = ['linear', 'rbf', 'poly', 'sigmoid']
# kernel_list = ['linear']
class_weights = [None, 'balanced']

In [14]:
def optimize(evals, trials, optimizer=tpe.suggest):
    space = {
        'kernel': hp.choice('kernel', kernel_list),
        'class_weight': hp.choice('class_weight', class_weights),
        'C': hp.uniform('C', 0, 20),
        'gamma': hp.uniform('gamma',0.001,10)
    }
    best = fmin(score, space, algo=optimizer, max_evals=evals, trials=trials)
    pbar.close()
    return best

In [15]:
train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []

def score(params):
    c = params['C']
    gamma = params['gamma']
    kernel = params['kernel']
    class_weight = params['class_weight']
    
    model = svm.SVC(kernel=kernel,
            C=c,
            random_state=42,
            decision_function_shape='ovr',
            # class_weight=self.class_weight,
            class_weight=class_weight,
            probability=True,
            gamma=gamma)
    
    model.fit(X_train, y_train)
    labels_prediction = model.predict(X_test)
    labels_probs = model.predict_proba(X_test)
    train_acc = accuracy_score(y_train, model.predict(X_train))
    train_loss = log_loss(y_train, model.predict_proba(X_train))
    test_acc = accuracy_score(y_test, labels_prediction)
    test_loss = log_loss(y_test, labels_probs)
    
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_acc)
    test_loss_list.append(test_loss)
    
    print('Parameters with this training accuracy {} and loss {} :'.format(train_acc, train_loss))
    print('Parameters with this testing accuracy {} and loss {} :'.format(test_acc, test_loss))
    print(params)
    pbar.update()
#     return {'loss': test_loss, 'status': STATUS_OK}
    return {'loss': -test_acc, 'status': STATUS_OK}

In [ ]:
import time
from tqdm import tqdm
import numpy as np

trials = Trials()
cores = 48
start = time.time()
evaluations = 1000
pbar = tqdm(total=evaluations, desc="Hyperopt")
best_param = optimize(evals=evaluations,
                      optimizer=tpe.suggest,
                      trials=trials)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)
end = time.time()
print('Time elapsed to optimize {0} executions: {1}'.format(evaluations, end - start))
best_param['kernel'] = kernel_list[best_param['kernel']]
best_param['class_weight'] = class_weights[best_param['class_weight']]
print('\n Best score:')
score(best_param)

Hyperopt:   0%|          | 0/1000 [00:00<?, ?it/s]


  0%|          | 0/1000 [00:00<?, ?it/s, best loss: ?]
Parameters with this training accuracy 0.996450467893 and loss 0.858198316055 :

  0%|          | 0/1000 [01:47<?, ?it/s, best loss: ?]
Parameters with this testing accuracy 0.56976744186 and loss 1.32180975031 :

  0%|          | 0/1000 [01:47<?, ?it/s, best loss: ?]
{'kernel': 'poly', 'C': 4.302310645441003, 'gamma': 7.729220738696893, 'class_weight': None}

  0%|          | 0/1000 [01:47<?, ?it/s, best loss: ?]

Hyperopt:   0%|          | 1/1000 [01:47<29:53:51, 107.74s/it]


  0%|          | 1/1000 [01:47<29:53:41, 107.73s/it, best loss: -0.56976744186]
Parameters with this training accuracy 0.996450467893 and loss 2.71860343194 :

  0%|          | 1/1000 [06:50<29:53:41, 107.73s/it, best loss: -0.56976744186]
Parameters with this testing accuracy 0.459302325581 and loss 1.94021705794 :

  0%|          | 1/1000 [06:50<29:53:41, 107.73s/it, best loss: -0.56976744186]
{'kernel': 'rbf', 'C': 19.74279514111955, 'gamma': 3.1259376627712605, 'class_weight': None}

  0%|          | 1/1000 [06:50<29:53:41, 107.73s/it, best loss: -0.56976744186]

Hyperopt:   0%|          | 2/1000 [06:50<46:06:29, 166.32s/it]

...


 40%|████      | 402/1000 [15:35:36<23:08:49, 139.35s/it, best loss: -0.668604651163]
Parameters with this training accuracy 0.993868989997 and loss 0.427536568368 :

 40%|████      | 402/1000 [15:37:26<23:08:49, 139.35s/it, best loss: -0.668604651163]
Parameters with this testing accuracy 0.668604651163 and loss 1.09740057925 :

 40%|████      | 402/1000 [15:37:26<23:08:49, 139.35s/it, best loss: -0.668604651163]
{'kernel': 'linear', 'C': 12.017029954347795, 'gamma': 9.324178667100222, 'class_weight': 'balanced'}

 40%|████      | 402/1000 [15:37:26<23:08:49, 139.35s/it, best loss: -0.668604651163]

Hyperopt:  40%|████      | 403/1000 [15:37:26<21:39:13, 130.58s/it]


 40%|████      | 403/1000 [15:37:26<21:39:13, 130.57s/it, best loss: -0.668604651163]
Parameters with this training accuracy 0.994837044208 and loss 0.425806116221 :

 40%|████      | 403/1000 [15:39:17<21:39:13, 130.57s/it, best loss: -0.668604651163]
Parameters with this testing accuracy 0.665697674419 and loss 1.09787365377 :

 40%|████      | 403/1000 [15:39:17<21:39:13, 130.57s/it, best loss: -0.668604651163]
{'kernel': 'linear', 'C': 13.263995466670272, 'gamma': 8.011900888670972, 'class_weight': 'balanced'}

 40%|████      | 403/1000 [15:39:17<21:39:13, 130.57s/it, best loss: -0.668604651163]

Hyperopt:  40%|████      | 404/1000 [15:39:17<20:37:50, 124.62s/it]


 40%|████      | 404/1000 [15:39:17<20:37:50, 124.62s/it, best loss: -0.668604651163]

In [ ]:
print('Time elapsed to optimize {0} executions: {1}'.format(evaluations, end - start))
print('\n Best score:')
score(best_param)

In [ ]:
# get the best params and train the model
SVC_opt = svm.SVC(kernel=best_param['kernel'],
            C=best_param['C'],
            random_state=42,
            decision_function_shape='ovr',
            # class_weight=self.class_weight,
            class_weight=best_param['class_weight'],
            probability=True,
            gamma=best_param['gamma'])


fitted_model = SVC_opt.fit(X=X_train,y=y_train)

In [ ]:
y_score = fitted_model.decision_function(X_test)
y_score

In [ ]:
test_acc = accuracy_score(y_test, fitted_model.predict(X_test))
test_acc

In [ ]:
from matplotlib import pyplot as plt 
%matplotlib inline 
x_list =range(len(train_acc_list))

plt.plot(x_list, train_acc_list, label='Training accuracy')
plt.plot(x_list, test_acc_list, label='Testing accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_list, train_loss_list, label='Training loss')
plt.plot(x_list, test_loss_list, label='Testing loss')
plt.legend()
plt.show()